<a href="https://colab.research.google.com/github/alphakilo11/Python/blob/main/wip_202208110948_Schroedinger_K13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Schrödinger Kapitel 13

In [1]:
import sqlite3
verbindung = sqlite3.connect("spam.db")
cursor = verbindung.cursor()
sql = '''CREATE TABLE eggs(
  cooler_typ TEXT,
  glückszahl INTEGER
);'''
cursor.execute(sql)
verbindung.commit()
verbindung.close()

In [2]:
# S 450
verbindung = sqlite3.connect("spam.db")
cursor = verbindung.cursor()
sql = '''INSERT INTO eggs(
  cooler_typ,
  glückszahl
)
VALUES("Schrödinger", 42), ('Hannes', 13), ('Lilly', 1024);'''
cursor.execute(sql)
verbindung.commit()
verbindung.close()

In [33]:
def erzeuge_tabellen(path, sql):
  """
  Creates a SQL db at given path and executes list of sql instructions
  demonstrated for CREATE TABLE and INSERT INTO
  """
  # establish database connection
  with sqlite3.connect(path) as verbindung:
    cursor = verbindung.cursor()



    for instruction in sql:
      # delete table if it already exists
      import re
      regex = re.compile('CREATE TABLE.*\(') # this might not find an empty table
      step = regex.findall(instruction)
      if len(step) > 0: # check if a CREATE TABLE instruction has been issued
        step = step[0][12:-1].strip()
        step = 'DROP TABLE IF EXISTS ' + step + ';'
        cursor.execute(step)
      # execute instructions
      cursor.execute(instruction)

In [4]:
foo = ['CREATE TABLE viren(name TEXT PRIMARY KEY, typ INTEGER, status TEXT);', '''
CREATE TABLE viren_typ(typ INTEGER PRIMARY KEY, groesse INT, signatur TEXT);''']
erzeuge_tabellen('strategie.db', foo)

In [5]:
foo = ['''
INSERT INTO viren_typ(typ, groesse, signatur)
VALUES(1,128,'ABAABA'),
(2, 256, 'ABAABA'),
(3,256,'BCCBCB')''', '''
INSERT INTO viren(name, typ, status)
VALUES ('T800', 1, 'aktiv'),
  ('T803', 2, 'aktiv'),
  ('Bit13', 3, 'aktiv'),
  ('Gorf13', 1, 'aktiv'),
  ('Gorf7', 2, 'aktiv')'''
  ]
erzeuge_tabellen('strategie.db', foo)

In [6]:
def sql_create_table_string(table_name, table_columns):
  """
  Constructs a SQL-String to create a table named table_name with columns provided in table_colums
  format: [column_name(str), data_type(str), primary_key(bool)]
  """
  step = '('
  for column in table_columns:
    step += column[0] + " " + column[1]
    if column[2] == True:
      step += ' PRIMARY KEY'
    step += ', '
  step = step[:-2] # remove the last comma
  step += ')'
  return 'CREATE TABLE ' + table_name + step + ';'

In [12]:
spam = [
    ['virus', 'TEXT', False],
    ['ort', 'TEXT', False],
    ['vorfall', 'TEXT', False]
]
erzeuge_tabellen('strategie.db', [sql_create_table_string('vorfall', spam)])

In [8]:
foo = ['''
INSERT INTO vorfall(virus, ort, vorfall)
VALUES
('T804','HD','NONE'),
("Gorf8", 'BIOS', 'Stoerung'),
('T805','RAM','NONE'),
('Gorf4', 'CPU', 'NONE')
'''
  ]
erzeuge_tabellen('strategie.db', foo)

In [39]:
def sql_create_input_string(table_name, table_columns, data):
  """
  Constructs a SQL-String to input data into a table table_name into columns provided in table_colums
  table_name(str), table_columns(str), data(iterable of strings)
  """
  col_string = '('
  for col in table_columns:
    col_string += col + ", "
  col_string = col_string[:-2] + ')'

  data_str = 'VALUES'
  for line in data:
    data_str += '('
    for value in line:
      data_str += "'" + value + "'" + ', '
    data_str = data_str[:-2] + '), ' 
  data_str = data_str[:-2] # remove the last comma
  return 'INSERT INTO ' + table_name + col_string + data_str

In [37]:
foo = (('T55', 'HD', 'NONE'), ('T64', 'BIOS', 'Stoerung'))
bar = sql_create_input_string('vorfall', ('virus', 'ort', 'vorfall'), foo)
erzeuge_tabellen('strategie.db', [bar])
del foo, bar

INSERT INTO vorfall(virus, ort, vorfall)VALUES('T55', 'HD', 'NONE'), ('T64', 'BIOS', 'Stoerung')


In [38]:
def sql_abfrage(path, sql):
  """Creates a SQL db at given path and executes list of sql instructions"""

  with sqlite3.connect(path) as verbindung:
    cursor = verbindung.cursor()
    foo = []
    for instruction in sql:
      cursor.execute(instruction)
      foo.append(cursor.fetchall())
  return foo

spam = sql_abfrage('strategie.db', ['SELECT * FROM viren_typ;', 'SELECT * FROM viren;', 'SELECT * FROM vorfall;'])
print(spam)
del spam

[[(1, 128, 'ABAABA'), (2, 256, 'ABAABA'), (3, 256, 'BCCBCB')], [('T800', 1, 'aktiv'), ('T803', 2, 'aktiv'), ('Bit13', 3, 'aktiv'), ('Gorf13', 1, 'aktiv'), ('Gorf7', 2, 'aktiv')], [('T800', 'HD', 'NONE'), ('Gorf7', 'BIOS', 'Stoerung'), ('T800', 'RAM', 'NONE'), ('Gorf3', 'CPU', 'NONE'), ('T72', 'HD', 'NONE'), ('T90', 'BIOS', 'Stoerung'), ('T55', 'HD', 'NONE'), ('T64', 'BIOS', 'Stoerung')]]
